# Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 기본 경로 (압축 해제 파일을 저장할 경로)
%cd /content/drive/MyDrive/Colab Notebooks/데이콘/2023포디블록

/content/drive/MyDrive/Colab Notebooks/데이콘/2023포디블록


In [ ]:
# # 압축 해제 필요하면 ctrl+/ 로 주석 지우기
# from google.colab import output

# # 기본 경로 (압축 해제 파일을 저장할 경로)
# %cd /content/drive/MyDrive/Colab Notebooks/데이콘/2022포디블록
# # 압축 해제
# !unzip 'open.zip'
# # 출력 결과 지우기
# output.clear()

## Import

In [3]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [5]:
CFG = {
    'IMG_SIZE':224, # Efficientnet B0 input size=224
    'EPOCHS':5,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':128,
    'SEED':41
}

## Fixed RandomSeed

In [6]:
# 토치
def seed_everything(seed):
    random.seed(seed) # transform에서 python의 random 사용하기 때문
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed) # 넘파이
    torch.manual_seed(seed) # torch.~ 시드 고정
    torch.cuda.manual_seed(seed) # type: ignore
    torch.backends.cudnn.deterministic = True # type: ignore
    torch.backends.cudnn.benchmark = True # type: ignore

seed_everything(CFG['SEED']) # Seed 고정

In [7]:
# # 텐서플로
# def seed_everything(seed):
#     random.seed(seed)
#     np.random.seed(seed)
#     os.environ["PYTHONHASHSEED"] = str(seed)
#     tf.random.set_seed(seed) # global seed

# seed_everything(CFG['SEED']) # Seed 고정

## Data Load

In [8]:
df = pd.read_csv('train.csv')

In [9]:
df.head()

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,./train/TRAIN_00000.jpg,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,./train/TRAIN_00001.jpg,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,./train/TRAIN_00002.jpg,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,./train/TRAIN_00003.jpg,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,./train/TRAIN_00004.jpg,1,0,0,0,0,0,0,0,0,0


## Train / Validation Split

In [10]:
# shuffle 후 split
df = df.sample(frac=1) # A random 100% sample of the DataFrame without replacement (default:replace=False)
train_len = int(len(df) * 0.8)

In [11]:
df.head()

,id,img_path,A,B,C,D,E,F,G,H,I,J
2253,TRAIN_02253,./train/TRAIN_02253.jpg,0,1,1,0,1,0,0,0,0,0
9609,TRAIN_09609,./train/TRAIN_09609.jpg,1,0,1,1,1,0,1,0,0,0
25434,TRAIN_25434,./train/TRAIN_25434.jpg,0,1,1,1,0,0,0,0,0,0
18725,TRAIN_18725,./train/TRAIN_18725.jpg,0,0,1,1,1,1,0,1,1,0
5887,TRAIN_05887,./train/TRAIN_05887.jpg,0,1,1,1,0,0,0,0,0,1


In [12]:
train = df[:train_len]
val = df[train_len:]

## Data Preprocessing

In [13]:
def get_labels(df):
    return df.iloc[:,2:].values # A~J

In [14]:
train_labels = get_labels(train)
val_labels = get_labels(val)

## CustomDataset

In [15]:
class CustomDataset(Dataset):
    # 전처리
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list # 이미지가 저장된 경로
        self.label_list = label_list # 인덱스+정답
        self.transforms = transforms
        
    # 특정 1개 샘플 가져오는 함수: dataset[i]로 i번째 샘플을 가져오는 인덱싱 위한 함수
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path) # 이미지, 라벨 불러오기, 아직 numpy
        
        # transform 적용: numpy to tensor
        if self.transforms is not None:
            image = self.transforms(image=image)['image'] # augmentation된 이미지 얻기
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
    
    # 데이터셋 길이를 리턴
    def __len__(self):
        return len(self.img_path_list)

In [16]:
# 이미지와 라벨 각각에 Augmentation을 적용하기 위한 객체/파이프라인을 생성합니다.
# 다양한 augmentation 방법을 A.Compose()로 묶어서 transform을 생성합니다.
# 이후 모델 학습 시 augmentation하기 위함

# import albumentations as A

train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225),
                max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
    ])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
    ])

'''
Resize: height, width 모두 224로 지정
Normalize: 입력 받은 이미지 값의 범위를 (0, 255) → (-1, 1) 범위로 줄여 1) 학습이 빨리 수렴되고 2) 특정 입력값의 영향 줄임
          보통 mean, std 값을 (0, 255) 사이의 값으로 저장하거나(이때 max_pixel_value=1.0)
          (-1, 1) 사이의 값으로 저장합니다.(이때 max_pixel_value=255)
          참고로 mean, std는 리스트, 튜플 또는 넘파이 배열로 입력하면 됩니다.
          
ToTensorV2: augmentation이 적용된 데이터를 numpy -> torch 타입으로 변환
'''

'\nResize: height, width 모두 224로 지정\nNormalize: 입력 받은 이미지 값의 범위를 (0, 255) → (-1, 1) 범위로 줄여 1) 학습이 빨리 수렴되고 2) 특정 입력값의 영향 줄임\n          보통 mean, std 값을 (0, 255) 사이의 값으로 저장하거나(이때 max_pixel_value=1.0)\n          (-1, 1) 사이의 값으로 저장합니다.(이때 max_pixel_value=255)\n          참고로 mean, std는 리스트, 튜플 또는 넘파이 배열로 입력하면 됩니다.\n          \nToTensorV2: augmentation이 적용된 데이터를 numpy -> torch 타입으로 변환\n'

In [18]:
# Dataset 정의
# 000_transform 이용해 실제 augmentation 적용
train_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
val_dataset = CustomDataset(val['img_path'].values, val_labels, test_transform)

# DataLoader 정의
# 순서 섞어서 batch_size(128)개씩 데이터 반환
train_loader = DataLoader(train_dataset, 
                          batch_size = CFG['BATCH_SIZE'], 
                          shuffle=False,
                          num_workers=0)
val_loader = DataLoader(val_dataset, 
                        batch_size = CFG['BATCH_SIZE'], 
                        shuffle=False, 
                        num_workers=0)

## Model Define

In [19]:
# nn.Module을 상속 받아 BaseModel 정의
class BaseModel(nn.Module):
    def __init__(self, num_classes=10):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x

## Train

In [20]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_acc = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
    
    return best_model

In [21]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)
    
    return _val_loss, _val_acc

## Run!!

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

  0%|          | 0/207 [00:00<?, ?it/s]

## Inference

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/12 [00:00<?, ?it/s]

## Submission

In [ ]:
submit = pd.read_csv('sample_submission.csv')

In [ ]:
submit.iloc[:,1:] = preds
submit.head()

,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,1,1,1,0,0,1,1,0,1,0
1,TEST_00001,0,1,1,0,1,1,1,0,0,0
2,TEST_00002,1,1,0,1,1,1,0,1,0,1
3,TEST_00003,1,1,0,0,1,1,1,0,1,0
4,TEST_00004,1,1,1,0,1,1,1,0,0,0


In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)